# Cross‑Analysis Recap — Notebook

This notebook scans a **results root folder** containing multiple experiment subfolders (e.g. `univ_delta_barthel`, `univ_theta_barthel`, `univ_exponent_barthel`, … — and the corresponding ones for FIM and Effectiveness).  
It aggregates the CSVs produced by your per‑experiment analysis (e.g. `metric_scores.csv`, `fold_scores.csv`, `feature_importances_mean.csv`, `predictions.csv`, `fold_predictions_long.csv`), compares **feature families** (periodic / aperiodic / complexity) across targets, tries to infer **ROIs/hemisphere** patterns, and generates a concise **Markdown report** + **plots** suitable for your supervisor.

## How to use
1. In the **Config** cell below, set `results_root` to your directory that contains all the sub‑analyses.  
2. Run the notebook top-to-bottom.  
3. Outputs (plots/CSVs/report) will be saved under `outdir` and previewed inline where possible.

In [1]:
import os, re, csv, json
from pathlib import Path
from typing import Dict, Any, List, Optional, Tuple, Union

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:

# ---------------- Helpers ----------------

def read_csv_rows(path: Path) -> List[Dict[str, str]]:
    if not path.exists():
        return []
    with open(path, "r", encoding="utf-8") as f:
        r = csv.DictReader(f)
        return [row for row in r]

def write_csv_rows(path: Path, rows: List[Dict[str, Any]], fieldnames: Optional[List[str]] = None):
    if not rows:
        return
    if fieldnames is None:
        fieldnames = list(rows[0].keys())
        extra = set().union(*[set(r.keys()) for r in rows]) - set(fieldnames)
        fieldnames += sorted(extra)
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=fieldnames)
        w.writeheader()
        for r in rows:
            w.writerow(r)

def safe_float(x, default=np.nan):
    try:
        return float(x)
    except Exception:
        return default

def best_metric_from_rows(metric_rows: List[Dict[str,str]]) -> Tuple[str, float, str]:
    """
    Choose the primary metric and value for comparison.
    Priority: R2 (max) -> RMSE (min) -> MAE (min) -> Accuracy (max) -> F1 (max).
    Returns (metric_name, score_value, direction) where direction in {"higher_is_better","lower_is_better"}.
    """
    m = {}
    for r in metric_rows:
        k = (r.get("metric") or "").lower()
        v = safe_float(r.get("value", np.nan))
        if k:
            m[k] = v
    candidates = []
    if "r2" in m:
        candidates.append(("r2", m["r2"], "higher_is_better"))
    if "rmse" in m:
        candidates.append(("rmse", m["rmse"], "lower_is_better"))
    if "mae" in m:
        candidates.append(("mae", m["mae"], "lower_is_better"))
    if "accuracy" in m:
        candidates.append(("accuracy", m["accuracy"], "higher_is_better"))
    if "f1" in m:
        candidates.append(("f1", m["f1"], "higher_is_better"))
    if candidates:
        return candidates[0]
    return ("", np.nan, "higher_is_better")

def infer_target_from_path(p: Path) -> str:
    s = p.as_posix().lower()
    if "barthel" in s:
        return "barthel"
    if "fim" in s:
        return "fim"
    if "effectiveness" in s:
        return "effectiveness"
    return "unknown"

def infer_family_from_path(p: Path) -> str:
    s = p.name.lower()
    if any(k in s for k in ["delta","theta"]):
        return "periodic"
    if any(k in s for k in ["exponent","offset"]):
        return "aperiodic"
    if any(k in s for k in ["lziv","higuci","higuchi"]):
        return "complexity"
    return "other"

def infer_feature_key_from_path(p: Path) -> str:
    s = p.name.lower()
    for k in ["delta","theta","exponent","offset","lziv","higuci","higuchi"]:
        if k in s:
            return k
    return "unknown"

def infer_roi_from_importances(imp_rows: List[Dict[str,str]]) -> Optional[str]:
    if not imp_rows:
        return None
    feats = [r.get("feature","") for r in imp_rows if r.get("feature")]
    text = "_".join(feats).lower()
    rois = ["frontal","temporal","parietal","occipital","central","cingulate","hippocampus","insula"]
    sides = ["left","right","l","r"]
    found_roi = None
    for roi in rois:
        if roi in text:
            found_roi = roi
            break
    found_side = None
    for s in sides:
        import re as _re
        if _re.search(r"(^|[_\-])" + s + r"([_\-]|$)", text):
            found_side = s
            break
    if found_roi and found_side:
        return f"{found_roi}_{found_side}"
    if found_roi:
        return found_roi
    return None

def infer_hemisphere(label: str) -> Optional[str]:
    if not label:
        return None
    s = label.lower()
    toks = s.split("_")
    if "right" in toks or "r" in toks:
        return "right"
    if "left" in toks or "l" in toks:
        return "left"
    return None

def choose_score_for_comparison(metric_name: str, value: float, direction: str) -> float:
    if np.isnan(value):
        return np.nan
    if direction == "lower_is_better":
        return -value
    return value


In [3]:
# --- PATCH: supporto ai file 'metrics_by_fold.csv' & 'feature_importances_mean_std.csv' ---

from pathlib import Path
import csv, numpy as np

EXPECTED_ANY = {
    "metric_scores.csv",
    "fold_scores.csv",
    "feature_importances_mean.csv",
    "predictions.csv",
    "fold_predictions_long.csv",
    # nuovi nomi visti nei tuoi run:
    "metrics_by_fold.csv",
    "feature_importances_mean_std.csv",
    "feature_importances_by_fold.csv",
}

def _read_csv_rows(path: Path):
    if not path.exists():
        return []
    with open(path, "r", encoding="utf-8") as f:
        return list(csv.DictReader(f))

def _coerce_metric_rows_from_metrics_by_fold(exp_dir: Path):
    """
    Converte 'metrics_by_fold.csv' in una lista tipo metric_scores.csv:
      [{'metric':'r2','value':...}, {'metric':'rmse','value':...}, ...]
    Prende la media sui fold per le colonne standard se presenti.
    """
    p = exp_dir / "metrics_by_fold.csv"
    rows = _read_csv_rows(p)
    if not rows:
        return []

    # Prova a capire quali colonne metriche sono presenti
    # (supportiamo nomi comuni; aggiungi qui eventuali varianti tue)
    candidates = ["r2","rmse","mae","mse","accuracy","f1","mape"]
    metrics_present = [m for m in candidates if m in rows[0].keys()]
    out = []
    for m in metrics_present:
        vals = []
        for r in rows:
            try:
                vals.append(float(r[m]))
            except Exception:
                pass
        if len(vals):
            out.append({"metric": m, "value": float(np.mean(vals))})
    return out

def _coerce_fold_scores_from_metrics_by_fold(exp_dir: Path):
    """
    Converte 'metrics_by_fold.csv' in un formato simile a fold_scores.csv:
      [{'fold':0, 'r2':..., 'rmse':...}, ...]
    Se non c'è una colonna fold, genera un indice.
    """
    p = exp_dir / "metrics_by_fold.csv"
    rows = _read_csv_rows(p)
    if not rows:
        return []
    # prova a usare 'fold' se esiste, altrimenti enumerazione
    has_fold = "fold" in rows[0]
    out = []
    for i, r in enumerate(rows):
        row = {"fold": int(r["fold"]) if has_fold and r.get("fold","").isdigit() else i}
        for k, v in r.items():
            if k.lower() == "fold":
                continue
            try:
                row[k.lower()] = float(v)
            except Exception:
                pass
        out.append(row)
    return out

def _coerce_imp_rows_from_mean_std(exp_dir: Path):
    """
    Converte 'feature_importances_mean_std.csv' in formato:
      [{'feature':..., 'mean_importance':..., 'std_importance':...}, ...]
    Supporta colonne: feature, (mean|mean_importance), (std|std_importance).
    """
    p = exp_dir / "feature_importances_mean_std.csv"
    rows = _read_csv_rows(p)
    if not rows:
        return []

    # Riconosci nomi colonne flessibili
    def pick(colnames, keys):
        keys_low = {k.lower(): k for k in keys}
        for c in colnames:
            if c in keys_low:
                return keys_low[c]
        return None

    keys = rows[0].keys()
    c_feat = pick(["feature","features","name"], keys)
    c_mean = pick(["mean_importance","mean","avg"], keys)
    c_std  = pick(["std_importance","std","sd"], keys)

    out = []
    for r in rows:
        feat = r.get(c_feat, "") if c_feat else ""
        mi = r.get(c_mean, "")
        sd = r.get(c_std, "")
        try:
            mi = float(mi)
        except Exception:
            mi = np.nan
        try:
            sd = float(sd)
        except Exception:
            sd = np.nan
        if feat:
            out.append({"feature": feat, "mean_importance": mi, "std_importance": sd})
    return out

def summarize_experiment(exp_dir: Path) -> dict:
    """
    Versione robusta che usa:
      - metric_scores.csv se presente, altrimenti metrics_by_fold.csv
      - fold_scores.csv se presente, altrimenti metrics_by_fold.csv
      - feature_importances_mean.csv se presente, altrimenti feature_importances_mean_std.csv
    """
    # --- metriche ---
    metric_rows = _read_csv_rows(exp_dir / "metric_scores.csv")
    if not metric_rows:
        metric_rows = _coerce_metric_rows_from_metrics_by_fold(exp_dir)

    # --- fold scores ---
    fold_rows = _read_csv_rows(exp_dir / "fold_scores.csv")
    if not fold_rows:
        fold_rows = _coerce_fold_scores_from_metrics_by_fold(exp_dir)

    # --- importances ---
    imp_rows = _read_csv_rows(exp_dir / "feature_importances_mean.csv")
    if not imp_rows:
        imp_rows = _coerce_imp_rows_from_mean_std(exp_dir)

    # (Questi due spesso non ci sono nelle tue cartelle ROI)
    pred_rows  = _read_csv_rows(exp_dir / "predictions.csv")
    fold_long  = _read_csv_rows(exp_dir / "fold_predictions_long.csv")

    # --- inferenze standard del notebook ---
    target = infer_target_from_path(exp_dir)
    family = infer_family_from_path(exp_dir)
    feature_key = infer_feature_key_from_path(exp_dir)
    roi_guess = infer_roi_from_importances(imp_rows) if imp_rows else None

    # Scegli metrica primaria
    metric_name, metric_value, direction = best_metric_from_rows(metric_rows)

    # Fallback R2 dai fold_long se non abbiamo metriche
    if not metric_name and fold_long:
        by_fold = {}
        for r in fold_long:
            f = int(r.get("fold", -1))
            by_fold.setdefault(f, {"y_true": [], "y_pred": []})
            try:
                by_fold[f]["y_true"].append(float(r.get("y_true", np.nan)))
                by_fold[f]["y_pred"].append(float(r.get("y_pred", np.nan)))
            except Exception:
                pass
        r2s = []
        for f, data in by_fold.items():
            y_t = np.asarray(data["y_true"], float)
            y_p = np.asarray(data["y_pred"], float)
            ss_res = np.nansum((y_t - y_p)**2)
            ss_tot = np.nansum((y_t - np.nanmean(y_t))**2)
            r2 = np.nan if ss_tot==0 else 1.0 - ss_res/ss_tot
            r2s.append(r2)
        if r2s:
            metric_name = "r2"
            metric_value = float(np.nanmean(r2s))
            direction = "higher_is_better"

    # Stima n per fold se possibile
    n_per_fold = 0
    if fold_rows and "fold" in fold_rows[0]:
        counts = {}
        for r in fold_rows:
            f = int(r.get("fold", -1))
            counts[f] = counts.get(f, 0) + 1
        if counts:
            n_per_fold = int(np.mean(list(counts.values())))

    # Top feature da importances
    top_feature = ""
    top_mean_imp = np.nan
    top_std_imp = np.nan
    if imp_rows:
        rows_sorted = sorted(imp_rows, key=lambda r: float(r.get("mean_importance", "nan")), reverse=True)
        if rows_sorted:
            top_feature = rows_sorted[0].get("feature","")
            top_mean_imp = float(rows_sorted[0].get("mean_importance", "nan") or np.nan)
            top_std_imp = float(rows_sorted[0].get("std_importance", "nan") or np.nan)

    return {
        "exp_dir": exp_dir.as_posix(),
        "target": target,
        "family": family,
        "feature_key": feature_key,
        "roi_guess": roi_guess or "",
        "metric_name": metric_name,
        "metric_value": metric_value,
        "metric_direction": direction,
        "score_for_compare": choose_score_for_comparison(metric_name, metric_value, direction) if metric_name else np.nan,
        "n_per_fold_test": n_per_fold,
        "top_feature": top_feature,
        "top_feature_mean_imp": top_mean_imp,
        "top_feature_std_imp": top_std_imp,
    }

def scan_results_root(root: Path) -> list[dict]:
    """
    Tollerante: considera 'esperimento' qualsiasi cartella che contenga
    ALMENO uno tra i file attesi (vecchi o nuovi).
    """
    experiments = set()
    for p in root.rglob("*.csv"):
        if p.name in EXPECTED_ANY or p.name.startswith("best_fold_") and p.name.endswith("_feature_importances.csv"):
            experiments.add(p.parent)
    summaries = [summarize_experiment(p) for p in sorted(experiments)]
    return summaries


In [7]:
results_root = Path("/Users/Altair93/Documents/Dottorato/PATHS/Python_analisys/Results/ML/results/univ_ROI")
outdir = results_root / "_recap"

print("Results root:", results_root)
print("Output dir  :", outdir)

Results root: /Users/Altair93/Documents/Dottorato/PATHS/Python_analisys/Results/ML/results/univ_ROI
Output dir  : /Users/Altair93/Documents/Dottorato/PATHS/Python_analisys/Results/ML/results/univ_ROI/_recap


In [ ]:
from pathlib import Path
import csv

results_root = Path("/Users/Altair93/Documents/Dottorato/PATHS/Python_analisys/Results/ML/results/univ_ROI")  # <-- conferma questo path

EXPECTED_ANY = {
    "metric_scores.csv",
    "fold_scores.csv",
    "feature_importances_mean.csv",
    "predictions.csv",
    "fold_predictions_long.csv",
    # i tuoi nomi reali:
    "metrics_by_fold.csv",
    "feature_importances_mean_std.csv",
    "feature_importances_by_fold.csv",
}

csvs = list(results_root.rglob("*.csv"))
print(f"CSV totali trovati: {len(csvs)}")
for p in csvs[:15]:
    print("-", p)

# Cartelle candidate esperimento: contengono almeno UN file tra quelli attesi
experiments = set()
for p in csvs:
    name = p.name.lower()
    if (name in {n.lower() for n in EXPECTED_ANY}) or (name.startswith("best_fold_") and name.endswith("_feature_importances.csv")):
        experiments.add(p.parent)

experiments = sorted(experiments)
print(f"\nCartelle esperimento trovate: {len(experiments)}")
for d in experiments[:20]:
    print("-", d)


In [9]:
summaries = scan_results_root(results_root)
print(f"Found {len(summaries)} experiment folders with CSVs.")

report_path = write_report(summaries, outdir)
print("Report written to:", report_path)

Found 0 experiment folders with CSVs.
Report written to: /Users/Altair93/Documents/Dottorato/PATHS/Python_analisys/Results/ML/results/univ_ROI/_recap/cross_analysis_recap.md


In [10]:
from pathlib import Path

# 1) Count and preview all CSVs under results_root
csvs = list(results_root.rglob("*.csv"))
print(f"Found {len(csvs)} CSV files under {results_root}")
for p in csvs[:10]:
    print("-", p)

# 2) Show which folders contain at least one of the expected CSV names
expected = {"metric_scores.csv","fold_scores.csv","feature_importances_mean.csv",
            "predictions.csv","fold_predictions_long.csv"}
exp_dirs = set()
for p in csvs:
    if p.name in expected:
        exp_dirs.add(p.parent)
print(f"\nExperiment-like folders found: {len(exp_dirs)}")
for d in list(sorted(exp_dirs))[:10]:
    print("-", d)


Found 71 CSV files under /Users/Altair93/Documents/Dottorato/PATHS/Python_analisys/Results/ML/results/univ_ROI
- /Users/Altair93/Documents/Dottorato/PATHS/Python_analisys/Results/ML/results/univ_ROI/univ_offset_effectiveness/metrics_by_fold.csv
- /Users/Altair93/Documents/Dottorato/PATHS/Python_analisys/Results/ML/results/univ_ROI/univ_offset_effectiveness/best_fold_2_feature_importances.csv
- /Users/Altair93/Documents/Dottorato/PATHS/Python_analisys/Results/ML/results/univ_ROI/univ_offset_effectiveness/feature_importances_by_fold.csv
- /Users/Altair93/Documents/Dottorato/PATHS/Python_analisys/Results/ML/results/univ_ROI/univ_offset_effectiveness/feature_importances_mean_std.csv
- /Users/Altair93/Documents/Dottorato/PATHS/Python_analisys/Results/ML/results/univ_ROI/univ_offset_barthel/metrics_by_fold.csv
- /Users/Altair93/Documents/Dottorato/PATHS/Python_analisys/Results/ML/results/univ_ROI/univ_offset_barthel/best_fold_5_feature_importances.csv
- /Users/Altair93/Documents/Dottorato/PA

In [17]:
rep = (outdir / "cross_analysis_recap.md")
if rep.exists():
    with open(rep, "r", encoding="utf-8") as f:
        print(f.read())
else:
    print("Report not found yet.")

Report not found yet.


In [ ]:
import csv, re, numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

outdir = results_root / "_recap"
outdir.mkdir(parents=True, exist_ok=True)

def read_csv_rows(path: Path):
    if not path.exists():
        return []
    with open(path, "r", encoding="utf-8") as f:
        return list(csv.DictReader(f))

def safe_float(x, default=np.nan):
    try: return float(x)
    except: return default

def infer_target_from_path(p: Path) -> str:
    s = p.as_posix().lower()
    if "barthel" in s: return "barthel"
    if "fim" in s: return "fim"
    if "effectiveness" in s: return "effectiveness"
    return "unknown"

def infer_family_from_path(p: Path) -> str:
    s = p.name.lower()
    if any(k in s for k in ["delta","theta"]): return "periodic"
    if any(k in s for k in ["exponent","offset"]): return "aperiodic"
    if any(k in s for k in ["lziv","higuci","higuchi"]): return "complexity"
    return "other"

def infer_feature_key_from_path(p: Path) -> str:
    s = p.name.lower()
    for k in ["delta","theta","exponent","offset","lziv","higuci","higuchi"]:
        if k in s: return k
    return "unknown"

def infer_roi_from_importances(imp_rows):
    if not imp_rows: return ""
    feats = [r.get("feature","") for r in imp_rows if r.get("feature")]
    text = "_".join(feats).lower()
    rois = ["frontal","temporal","parietal","occipital","central","cingulate","hippocampus","insula"]
    sides = ["left","right","l","r"]
    found_roi = next((roi for roi in rois if roi in text), None)
    found_side = None
    for s in sides:
        if re.search(rf"(^|[_\-]){s}([_\-]|$)", text):
            found_side = s; break
    if found_roi and found_side: return f"{found_roi}_{found_side}"
    return found_roi or ""

def choose_score_for_comparison(metric_name, value, direction):
    if value is None or np.isnan(value): return np.nan
    if direction == "lower_is_better": return -value
    return value

def coerce_metric_rows_from_metrics_by_fold(exp_dir: Path):
    rows = read_csv_rows(exp_dir / "metrics_by_fold.csv")
    if not rows: return []
    # cerca colonne comuni (case-insensitive)
    keys = {k.lower(): k for k in rows[0].keys()}
    candidates = ["r2","rmse","mae","mse","accuracy","f1","mape"]
    out = []
    for m in candidates:
        k = keys.get(m)
        if not k: continue
        vals = [safe_float(r.get(k)) for r in rows]
        vals = [v for v in vals if np.isfinite(v)]
        if vals:
            out.append({"metric": m, "value": float(np.mean(vals))})
    return out

def coerce_imp_rows_from_mean_std(exp_dir: Path):
    rows = read_csv_rows(exp_dir / "feature_importances_mean_std.csv")
    if not rows: return []
    keys = {k.lower(): k for k in rows[0].keys()}
    c_feat = keys.get("feature") or keys.get("features") or keys.get("name")
    c_mean = keys.get("mean_importance") or keys.get("mean") or keys.get("avg")
    c_std  = keys.get("std_importance")  or keys.get("std")  or keys.get("sd")
    out = []
    for r in rows:
        feat = r.get(c_feat, "") if c_feat else ""
        mi = safe_float(r.get(c_mean)) if c_mean else np.nan
        sd = safe_float(r.get(c_std)) if c_std else np.nan
        if feat:
            out.append({"feature": feat, "mean_importance": mi, "std_importance": sd})
    return out

def best_metric_from_rows(metric_rows):
    # priorità: R2 -> RMSE -> MAE -> Accuracy -> F1
    m = { (r.get("metric") or "").lower(): safe_float(r.get("value")) for r in metric_rows }
    if "r2" in m:      return ("r2", m["r2"], "higher_is_better")
    if "rmse" in m:    return ("rmse", m["rmse"], "lower_is_better")
    if "mae" in m:     return ("mae", m["mae"], "lower_is_better")
    if "accuracy" in m:return ("accuracy", m["accuracy"], "higher_is_better")
    if "f1" in m:      return ("f1", m["f1"], "higher_is_better")
    return ("", np.nan, "higher_is_better")

# 1) individua esperimenti (tollerante, usa i tuoi nomi)
EXPECTED_ANY = {"metrics_by_fold.csv","feature_importances_mean_std.csv","feature_importances_by_fold.csv",
                "metric_scores.csv","fold_scores.csv","feature_importances_mean.csv","predictions.csv","fold_predictions_long.csv"}

experiments = sorted({
    p.parent for p in results_root.rglob("*.csv")
    if (p.name.lower() in {n.lower() for n in EXPECTED_ANY}) or (p.name.lower().startswith("best_fold_") and p.name.lower().endswith("_feature_importances.csv"))
})

print("Esperimenti trovati:", len(experiments))
for d in experiments[:10]:
    print("-", d)

# 2) costruisci il summary per ciascun esperimento
summaries = []
for exp in experiments:
    metric_rows = coerce_metric_rows_from_metrics_by_fold(exp)
    imp_rows = coerce_imp_rows_from_mean_std(exp)

    target = infer_target_from_path(exp)
    family = infer_family_from_path(exp)
    feature_key = infer_feature_key_from_path(exp)
    roi_guess = infer_roi_from_importances(imp_rows)

    metric_name, metric_value, direction = best_metric_from_rows(metric_rows)
    score_for_compare = choose_score_for_comparison(metric_name, metric_value, direction) if metric_name else np.nan

    # Top feature
    top_feature = ""; top_mean_imp = np.nan; top_std_imp = np.nan
    if imp_rows:
        imp_sorted = sorted(imp_rows, key=lambda r: r["mean_importance"] if np.isfinite(r["mean_importance"]) else -1, reverse=True)
        if imp_sorted:
            top_feature = imp_sorted[0]["feature"]
            top_mean_imp = imp_sorted[0]["mean_importance"]
            top_std_imp = imp_sorted[0]["std_importance"]

    summaries.append({
        "exp_dir": exp.as_posix(),
        "target": target,
        "family": family,
        "feature_key": feature_key,
        "roi_guess": roi_guess or "",
        "metric_name": metric_name,
        "metric_value": metric_value,
        "metric_direction": direction,
        "score_for_compare": score_for_compare,
        "top_feature": top_feature,
        "top_feature_mean_imp": top_mean_imp,
        "top_feature_std_imp": top_std_imp,
    })

# 3) salva tabella
def write_csv_rows(path: Path, rows, fieldnames=None):
    if not rows: return
    if fieldnames is None:
        fieldnames = list(rows[0].keys())
        extra = set().union(*[set(r.keys()) for r in rows]) - set(fieldnames)
        fieldnames += sorted(extra)
    with open(path, "w", newline="", encoding="utf-8") as f:
        w = csv.DictWriter(f, fieldnames=fieldnames)
        w.writeheader()
        for r in rows:
            w.writerow(r)

write_csv_rows(outdir / "experiments_summary.csv", summaries)
print("Salvato:", outdir / "experiments_summary.csv")

# 4) grafici per target
def plot_best_by_family(summaries, target, outdir):
    rows = [s for s in summaries if s["target"]==target and np.isfinite(s["score_for_compare"])]
    if not rows: 
        print("[skip] best_by_family", target); 
        return
    best = {}
    for r in rows:
        fam = r["family"]
        if fam not in best or r["score_for_compare"] > best[fam]["score_for_compare"]:
            best[fam] = r
    labels = list(best.keys())
    vals = [best[k]["score_for_compare"] for k in labels]
    plt.figure(figsize=(7,5))
    x = np.arange(len(labels))
    plt.bar(x, vals)
    plt.xticks(x, labels)
    plt.ylabel("Comparable score (higher is better)")
    plt.title(f"Best by family — {target}")
    plt.tight_layout()
    plt.savefig(outdir / f"best_by_family_{target}.png", dpi=150)
    plt.show()

def plot_top_rois(summaries, target, outdir, top_k=12):
    rows = [s for s in summaries if s["target"]==target and np.isfinite(s["score_for_compare"])]
    if not rows: 
        print("[skip] top_rois", target); 
        return
    best_by_roi = {}
    for r in rows:
        roi = r["roi_guess"] or r["feature_key"]
        if roi not in best_by_roi or r["score_for_compare"] > best_by_roi[roi]["score_for_compare"]:
            best_by_roi[roi] = r
    items = sorted(best_by_roi.items(), key=lambda kv: kv[1]["score_for_compare"], reverse=True)[:top_k]
    labels = [k for k,_ in items]
    vals = [v["score_for_compare"] for _, v in items]
    plt.figure(figsize=(10, max(4, 0.5*len(labels))))
    x = np.arange(len(labels))
    plt.bar(x, vals)
    plt.xticks(x, labels, rotation=45, ha="right")
    plt.ylabel("Comparable score (higher is better)")
    plt.title(f"Top ROIs — {target}")
    plt.tight_layout()
    plt.savefig(outdir / f"top_rois_{target}.png", dpi=150)
    plt.show()

def hemisphere_from_label(label: str):
    s = (label or "").lower().split("_")
    if "right" in s or "r" in s: return "right"
    if "left" in s or "l" in s: return "left"
    return None

def plot_hemisphere_summary(summaries, target, outdir):
    rows = [s for s in summaries if s["target"]==target and np.isfinite(s["score_for_compare"])]
    if not rows:
        print("[skip] hemisphere", target)
        return
    left = []; right = []
    for r in rows:
        hemi = hemisphere_from_label(r.get("roi_guess",""))
        if hemi == "left": left.append(r["score_for_compare"])
        elif hemi == "right": right.append(r["score_for_compare"])
    labels = ["left","right"]
    vals = [float(np.nanmean(left)) if left else 0.0,
            float(np.nanmean(right)) if right else 0.0]
    plt.figure(figsize=(6,5))
    x = np.arange(len(labels))
    plt.bar(x, vals)
    plt.xticks(x, labels)
    plt.ylabel("Mean comparable score")
    plt.title(f"Hemisphere summary — {target}")
    plt.tight_layout()
    plt.savefig(outdir / f"hemisphere_summary_{target}.png", dpi=150)
    plt.show()

targets = sorted(set(s["target"] for s in summaries))
for t in targets:
    plot_best_by_family(summaries, t, outdir)
    plot_top_rois(summaries, t, outdir)
    plot_hemisphere_summary(summaries, t, outdir)

# 5) report markdown
md = []
md.append("# Cross-analysis Recap\n")
md.append("Confronto tra **periodic (delta/theta)**, **aperiodic (exponent/offset)** e **complexity (lziv/higuci)** sui target (Barthel, FIM, Effectiveness).")
md.append("Gli score sono resi comparabili come *higher is better* (MAE/RMSE negati).\\n")

for t in targets:
    rows_t = [s for s in summaries if s["target"]==t and np.isfinite(s["score_for_compare"])]
    md.append(f"## Target: {t}\\n")
    if not rows_t:
        md.append("_Nessun esperimento valido._\\n")
        continue
    by_fam = {}
    for r in rows_t:
        fam = r["family"]
        if fam not in by_fam or r["score_for_compare"] > by_fam[fam]["score_for_compare"]:
            by_fam[fam] = r
    md.append("**Best by family**:\\n")
    for fam, r in by_fam.items():
        md.append(f"- {fam}: `{r['metric_name']}={r['metric_value']:.4f}` @ ROI `{r['roi_guess'] or r['feature_key']}` (dir={r['metric_direction']})")
    md.append("")
    best_by_roi = {}
    for r in rows_t:
        roi = r["roi_guess"] or r["feature_key"]
        if roi not in best_by_roi or r["score_for_compare"] > best_by_roi[roi]["score_for_compare"]:
            best_by_roi[roi] = r
    top5 = sorted(best_by_roi.items(), key=lambda kv: kv[1]["score_for_compare"], reverse=True)[:5]
    md.append("**Top ROIs overall**:\\n")
    for i,(roi, rr) in enumerate(top5, 1):
        md.append(f"{i}. `{roi}` — {rr['metric_name']}={rr['metric_value']:.4f} ({rr['family']})")
    md.append("")
    md.append(f"![Best by family — {t}](best_by_family_{t}.png)\\n")
    md.append(f"![Top ROIs — {t}](top_rois_{t}.png)\\n")
    md.append(f"![Hemisphere summary — {t}](hemisphere_summary_{t}.png)\\n")

report_path = outdir / "cross_analysis_recap.md"
with open(report_path, "w", encoding="utf-8") as f:
    f.write("\n".join(md))

print("Report scritto in:", report_path)
